In [3]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt

from pytensor.graph import Apply, Op
from scipy.optimize import approx_fprime

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
RANDOM_SEED = 42

def stick_breaking(beta):
    portion_remaining = pt.concatenate([[1], pt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * portion_remaining

def reparameterize(pi):
    return pi / pi.sum()

def hierarchical_beta(alpha0, beta, nsources, k):
    """Hierarchical Beta distribution for multiple sources."""
    pi_tilt_sources = []
    
    for s in range(nsources):
        beta_params = [(alpha0 * beta[k], alpha0 * (1 - pm.math.sum(beta[:k + 1]))) for k in range(k)]
        pi_tilt = pm.Beta(f'pi_tilt_{s}', 
                          alpha=[b[0] for b in beta_params], 
                          beta=[b[1] for b in beta_params], 
                          dims="component")
        pi_tilt_sources.append(pi_tilt)
    
    #return pm.math.stack(pi_tilt_sources, axis=0)
    return pi_tilt_sources

In [ ]:
import numpy as np
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm
import pandas as pd

# Define interpolate function
def interpolate(x0, y0, x):
    x = np.array(x)
    idx = np.searchsorted(x0, x)
    dl = np.array(x - x0[idx - 1])
    dr = np.array(x0[idx] - x)
    d = dl + dr
    wl = dr / d
    return wl * y0[idx - 1] + (1 - wl) * y0[idx]

# Load data and ensure it's a NumPy array
data = pd.read_csv('grids_example_1.csv', header=None).values
param_min = data[0, :].min()
param_max = data[0, :].max()

# Function to compute log-likelihood    
def my_loglike(x, data):
    x_vals = data[0, :]
    loglike_vals = data[1, :]
    return interpolate(x_vals, loglike_vals, x)

# Define custom PyTensor operation
class LogLike(Op):
    def make_node(self, x, data):
        x = pt.as_tensor_variable(x)  # Ensure x is a tensor
        data = pt.as_tensor_variable(data)  # Ensure data is a tensor
        inputs = [x, data]
        outputs = [pt.TensorType(dtype="float64", broadcastable=(False,))()]
        return Apply(self, inputs, outputs)

    def perform(self, node, inputs, outputs):
        x, data = inputs
        x_vals = data[0, :]
        loglike_vals = data[1, :]
        loglike_eval = interpolate(x_vals, loglike_vals, x)
        outputs[0][0] = np.array(loglike_eval)

# Initialize operation
loglike_op = LogLike()

# PyMC model without observed data
with pm.Model() as no_grad_model:
    # Define prior for x
    x = pm.Uniform("x", lower=param_min, upper=param_max, shape=1)

    # Add a custom potential for the likelihood
    pm.Potential("likelihood", loglike_op(x, data))

    # Sample posterior
    idata = pm.sample(
        200000, 
        tune=5000,
        chains=8,
    )

In [ ]:
import numpy as np
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm
import pandas as pd
import matplotlib.pyplot as plt

# Define interpolate function
def interpolate(x0, y0, x):
    x = np.array(x)
    idx = np.searchsorted(x0, x)
    dl = np.array(x - x0[idx - 1])
    dr = np.array(x0[idx] - x)
    d = dl + dr
    wl = dr / d
    return wl * y0[idx - 1] + (1 - wl) * y0[idx]

# Load data and ensure it's a NumPy array
data = pd.read_csv('grids_example_1.csv', header=None).values
param_min = data[0, :].min()
param_max = data[0, :].max()

In [ ]:
# plot the data
plt.plot(data[0, :], data[1, :], 'o')

plt.show()

### **1. Interpolation of Log-Likelihoods**
For each outcome \( j \), we interpolate the log-likelihood for each component \( k \):

$$\hat{\ell}_{j,k} = \text{interpolate}(x_{\text{grid},j}, \ell_{\text{grid},j}, \beta_{j,k})$$


$$\ell_k = \sum_{j=1}^{N_{\text{outcomes}}} \hat{\ell}_{j,k}$$

$$\ell_{\text{final}} = \log \sum_{k=1}^{N_{\text{components}}} \pi_k e^{\ell_k}$$


In [16]:
import numpy as np
import pandas as pd
import pytensor.tensor as pt
from pytensor.graph.op import Op, Apply
import pymc as pm

# Load profile likelihood data
data = pd.read_csv("profileLikelihoods_NCs_long.csv")

# Define interpolation function
def interpolate(x0, y0, x):
    x = np.array(x)
    idx = np.searchsorted(x0, x)
    dl = np.array(x - x0[idx - 1])
    dr = np.array(x0[idx] - x)
    d = dl + dr
    wl = dr / d
    return wl * y0[idx - 1] + (1 - wl) * y0[idx]

# Define custom likelihood function
class LogLike(Op):
    def make_node(self, β, num_outcomes, source_data):
        β = pt.as_tensor(β)  # Ensure scalar
        num_outcomes = pt.as_tensor(int(num_outcomes))  # Ensure scalar integer
        source_data = pt.as_tensor(np.asarray(source_data))  # Ensure NumPy array

        # The output must be a single scalar
        outputs = [pt.dscalar()]
        return Apply(self, [β, num_outcomes, source_data], outputs)

    def perform(self, node, inputs, outputs):
        β, num_outcomes, source_data = inputs
        num_outcomes = int(num_outcomes)  # Ensure integer

        # Compute total log-likelihood (scalar β)
        total_log_likelihood = 0.0

        for j in range(num_outcomes):
            # Extract outcome-specific x_vals and loglike_vals
            outcome_data = source_data[source_data[:, 2] == (j+1)]  # Filter by outcome index
            
            if outcome_data.shape[0] == 0:
                raise ValueError(f"No data found for outcome {j+1} in source_data!")

            x_vals = outcome_data[:, 0]  # Parameter grid points
            loglike_vals = outcome_data[:, 1]  # Log-likelihood values

            # Interpolate the log-likelihood for β
            loglike_j = interpolate(x_vals, loglike_vals, β)

            # Sum in log-space
            total_log_likelihood += loglike_j

        # Ensure the output is a scalar
        outputs[0][0] = np.array(total_log_likelihood)

# Initialize the custom operation
loglike_op = LogLike()

# Simulated test case for source 1
n_outcomes_test = int(data["outcome"].nunique())  # Ensure integer
n_components = 3  # Assume 3 components for illustration

np.random.seed(42)
β_test = np.random.uniform(low=-1, high=-0.5)  # Simulated β values
num_outcomes_test = int(data[data["source"] == 1]["outcome"].nunique())  # Ensure integer
source_1_data = data[data["source"] == 1][["point", "value", "outcome"]].values  # Include outcome column
weights_test = np.full(n_components, 1 / n_components)  # Equal mixture weights


test_out = loglike_op(β_test, num_outcomes_test, source_1_data)
print("Evaluated Test Likelihood Output:", test_out.eval())


Evaluated Test Likelihood Output: -30.351530816420613


In [6]:
np.random.seed(42)
β_test = np.random.uniform(low=-1, high=-0.5)  # Simulated β values
num_outcomes_test = int(data[data["source"] == 1]["outcome"].nunique())  # Ensure integer
source_1_data = data[data["source"] == 1][["point", "value", "outcome"]].values  # Include outcome column
weights_test = np.full(n_components, 1 / n_components)  # Equal mixture weights

In [17]:
def stick_breaking(beta):
    portion_remaining = pt.concatenate([[1], pt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * portion_remaining

def reparameterize(pi):
    return pi / pi.sum()

def hierarchical_beta(alpha0, beta, nsources, k):
    """Hierarchical Beta distribution for multiple sources."""
    pi_tilt_sources = []
    
    for s in range(nsources):
        beta_params = [(alpha0 * beta[k], alpha0 * (1 - pm.math.sum(beta[:k + 1]))) for k in range(k)]
        pi_tilt = pm.Beta(f'pi_tilt_{s}', 
                          alpha=[b[0] for b in beta_params], 
                          beta=[b[1] for b in beta_params], 
                          dims="component")
        pi_tilt_sources.append(pi_tilt)
    
    #return pm.math.stack(pi_tilt_sources, axis=0)
    return pi_tilt_sources

In [ ]:
import pymc as pm
import numpy as np
import pandas as pd

# Load profile likelihood data
data = pd.read_csv("profileLikelihoods_NCs_long.csv")

# Constants
N_sources = data["source"].nunique()  # Number of sources
k = 5  # Number of mixture components

# Precompute source data and number of outcomes
source_data_dict = {}
num_outcomes_dict = {}

for s in range(1, N_sources + 1):  # Ensure correct indexing
    source_data_dict[s] = data[data["source"] == s][["point", "value", "outcome"]].astype(float).values
    num_outcomes_dict[s] = int(data[data["source"] == s]["outcome"].nunique())

# PyMC Model
with pm.Model(coords={"component": np.arange(k), "n_source": np.arange(N_sources)}) as HDP_model:

    # Priors for hierarchical Dirichlet process weights
    gamma = pm.Gamma("gamma", 1.0, 5.0)
    alpha0 = pm.Gamma("alpha0", 1.0, 5.0)
    
    # Stick-breaking process for mixture weights
    beta_tilt = pm.Beta("beta_tilt", 1.0, gamma, dims="component")
    beta = pm.Deterministic("beta", stick_breaking(beta_tilt), dims="component")
    π_tilt = hierarchical_beta(alpha0, beta, nsources=N_sources, k=k)

    π_norms = []
    for s in range(N_sources):
        π = pm.Deterministic(f"π_{s}", stick_breaking(π_tilt[s]), dims="component")
        π_norms.append(pm.Deterministic(f"π_norm_{s}", reparameterize(π), dims=["component"]))

    # Mixture component parameters
    μ = pm.Normal(
        "μ",
        mu=data["point"].mean(),  # Use the mean of the profile likelihood grid points
        sigma=10,
        shape=k,
        transform=pm.distributions.transforms.ordered,  # Ensure μ is ordered
        initval=np.linspace(-1, 1, k),  # Reasonable ordered starting values
    )

    σ = pm.HalfNormal("σ", sigma=10, shape=k)

    # Likelihood for each source
    for s in range(1, N_sources + 1):  # Ensure correct indexing
        # Draw β from the mixture model
        β = pm.NormalMixture(f'β_{s}', w=π_norms[s-1], mu=μ, sigma=σ)
        βt = pm.Deterministic(f"β_clipped_{s}", pm.math.clip(β, source_data[:, 0].min(), source_data[:, 0].max()))
        #βt = pm.Truncated(f"β_truncated_{s}", β , lower=source_data[:, 0].min(), upper=source_data[:, 0].max())
        # Use precomputed values
        num_outcomes = num_outcomes_dict[s]
        source_data = source_data_dict[s]

        likelihood = pm.Potential(f"loglike_{s}", loglike_op(βt, num_outcomes, source_data))

    # Sampling
    trace = pm.sample(
        tune=5000,
        draws=20000,
        init="advi",
        #step=pm.Metropolis(),  # Use Metropolis for non-differentiable likelihood
        random_seed=42,
    )

